![self_station](./webcrawling/self_station.jpg)

- 운전을 하면 필연적으로 기름가격(유가)에 대해 관심을 가질 수 밖에 없습니다.


- 특히 서울시내에는 생각보다 주유소가 많지 않고, 교통체증이 심해서 자주 가는 주유소를 많이 방문합니다.


- 주유소 중에서 소폭의 가격인하를 하기 위해 알바가 없고 셀프주유를 하는 곳이 많이 생겼습니다.


- 과연 셀프 주유소는 가격이 저렴한지 궁금해졌습니다.


- 전국 주유소 가격을 제공하는 Opinet에서 크롤링을 통해 전국 주유소 가격을 얻은 뒤, 셀프 주유소의 가격에 대해 분석해봅니다.

http://www.opinet.co.kr/user/main/mainView.do

## 2.1. Opinet 웹페이지 분석 

![opinet](./webcrawling/opinet.png)

- Opinet에서 크롤링을 하려고 웹페이지 분석을 합니다.


- Opinet은 유가 정보를 동적 웹페이지 환경에서 제공합니다. (말그대로 살아 움직이는 페이지를 동적 웹페이지라고 합니다.)


- 왼쪽 navigation tab에서 지역을 설정할 수 있으며, 해당 지역을 "조회" 버튼을 눌러 데이터를 웹페이지에 불러온 뒤에 좌측 하단에 있는 "엑셀저장" 버튼을 누르면 깔끔하게 정리된 .xls 파일을 다운로드 받을 수 있습니다.


- 엑셀저장 기능을 빼고 bs4를 이용해서 왼쪽에 나타낸 데이터를 긁어올 수도 있지만, 이번에는 더 편하게 데이터를 받을 수 있는 엑셀저장을 활용해보겠습니다.


- 그런데 여기서 몇 가지 문제가 발생하는데, bs4만으로는 tag를 불러올 수가 없습니다.


**1) 지역을 다르게 눌러 조회를 해봐도 URL이 바뀌지 않습니다.**


**2) 실제로 bs4를 통해 좌측 데이터를 크롤링해도 긁어와지지 않습니다. (어떠한 방법으로 blocking 해놓은듯 합니다.)**


**3) bs4를 이용해서 지역을 바꿔서 설정할 수 없습니다. URL 규칙에 이 부분이 적용되지 않는 동적페이지이기 때문입니다.**

# Selenium with Chromedriver

- 이와 같은 동적페이지를 제어할 수 있는 라이브러리와 도구가 있습니다.


- 그것이 바로 구글에서 만든 Chromedriver와 selenium 입니다.

https://chromedriver.chromium.org/downloads

![chromedriver](./webcrawling/chromedriver.png)

- chromedriver는 구글이 ChromeOS를 개발하면서 만든 도구입니다.


- 가상의 chrome 환경을 만들어서 프로그램을 통해 web browser를 자동제어할 수 있게 해줍니다.


- selenium 같은 라이브러리를 사용하여 제어합니다.


- 위의 사이트에서 현재 사용하는 Chrome에 맞는 version을 다운로드 받습니다.

![selenium](./webcrawling/selenium.png)

- selenium은 코드에 의해 제어되는 chromedriver를 사용할 수 있게 해주는 오픈소스 라이브러리입니다.

In [ ]:
%pip install selenium

In [8]:
# selenium을 통해 webdriver를 제어하기 위해서 라이브러리를 불러옵니다.

from selenium import webdriver

# 동적페이지 크롤링에서는 페이지 로딩을 확인하기 위해서 기다리는 시간이 필요합니다.

webdriver.Chrome()



<selenium.webdriver.chrome.webdriver.WebDriver (session="782eeb55b45d7dc57de093ec8e3d8fe6")>

## 2.2. 전국 지역별 유가 정보 얻기

- selenium을 사용할 때 해야되는 기본적인 함수가 2가지 있습니다.


- webdriver.Chrome()과 driver.get() 입니다.


- webdriver.Chrome()은 실제 chromedriver가 존재하는 위치에서 chromedriver를 실행할 수 있게 불러오는 함수입니다.


- driver.get()은 selenium이 제어하고 있는 chromedriver가 해당 URL로 접속하게 하는 함수입니다.


- 여기서 또 문제점이 발생합니다. 지역별 검색 페이지에 접속해서 해당 URL로 driver.get()을 시도해도 메인페이지로 접속이 되는 문제입니다. 이런 경우는 해당 웹페이지 관리자가 다른 subpage들도 direct로 접속하지 못하게 메인페이지로 자동 redirection을 만들어 놓은 경우입니다.


- 이를 해결하기 위해 메인페이지에 먼저 접속한 뒤에, 지역별 페이지로 들어갑니다.


- Opinet 코드를 살펴보니 이 기능이 javascript 함수를 호출하는 것으로 되어 있습니다.


- selenium에서는 execute_script() 함수가 이를 지원하기 때문에 execute_script("goSubPage(0,0,99)")를 이용하여 해당 페이지에 접속합니다.

![redirection](./webcrawling/opinet_redirection.png)

In [1]:
# 해당 경로에 있는 chromedriver를 불러옵니다.
from selenium import webdriver
import time

# opinet.co.kr에 접속합니다.
driver = webdriver.Chrome()
driver.get('https://www.opinet.co.kr/searRgSelect.do')

# 2초 쉽니다. (페이지 로딩을 위해)
time.sleep(2)

# 메인 페이지에서 유가 가격 검색 페이지로 이동합니다. javascript를 활용합니다.
# <a href="javascript:goSubPage(0,0,99);"><span>지역별</span></a>
driver.execute_script('goSubPage(0,0,99)')

- selenium에서 특정 웹페이지 원소를 접근하는 방법이 여러가지가 있습니다.


**1) find_element_by_tag_name**


**2) find_element_by_selector**


**3) find_element_by_xpath**


**4) find_element_by_id**


각자 취향에 맞게 선택할 수 있습니다. 때에 따라 CSS selector만 선택해야 할때가 있고, xpath만 선택해야할 때가 있는데 이는 복잡한 문제이므로 넘어가겠습니다.

보통은 xpath로 일단 시도해본 뒤, 안되면 CSS selector나 tag name으로 시도해보면 좋습니다.


- 우선 우리는 지역목록을 가져와야 합니다.


- 이는 페이지에 drop-down 메뉴를 선택하여 해당 목록을 가져온 뒤, 하나하나 눌러서 접속해야 합니다.


- 지역에서 처음에 "도"를 고르고 그 뒤에 "구"를 고른 뒤에 "조회" 버튼을 누르면 해당 정보를 가져올 수 있습니다


- 정보를 제대로 가져왔는지는 가운데 있는 지도가 제대로 로드가 되었는지 확인해보면 됩니다. (초반 몇 개만)

![get_price](./webcrawling/opinet_get_price.png)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
# 앞서 배웠던 방식으로 xpath를 얻어옵니다

sido = driver.find_element(By.XPATH,'//*[@id="SIDO_NM0"]')
sido_names = sido.find_elements(By.TAG_NAME,'option')
# 앞서 배웠던 방식으로 tag을 찾아서 가져옵니다.

sido_list = []
for sido_name in sido_names:
    sido_list.append(sido_name.get_attribute('value'))
# tag에서 value들을 하나씩 뽑아냅니다.

sido_list = sido_list[1:]
sido_list


['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [4]:
sido = driver.find_element(By.XPATH,('//*[@id="SIDO_NM0"]'))
sido.send_keys('서울특별시')

- 이제 시도에 대한 정보를 얻었으니, 시/군/구도 동일한 방식으로 얻어오도록 합니다!

In [5]:
# 시/군/구 목록 수집

sigungu = driver.find_element(By.XPATH,'//*[@id="SIGUNGU_NM0"]')
sigungu_names = sigungu.find_elements(By.TAG_NAME,'option')

sigungu_list = []

for sigungu_name in sigungu_names:
    sigungu_list.append(sigungu_name.get_attribute('value'))

sigungu_list = sigungu_list[1:]
sigungu_list


['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [42]:
sigungu.send_keys('강서구')

In [35]:
# "조회" 버튼의 xpath를 찾아서 클릭합니다.

driver.find_element(By.XPATH,'//*[@id="searRgSelect"]').click()

In [44]:
# "엑셀저장" 버튼의 xpath를 찾아서 클릭합니다.

driver.find_element(By.XPATH,'//*[@id="glopopd_excel"]').click()

# 엑셀 파일이 저장 된다.

### 모두 종합해서 하나의 코드로 만들어봅시다!

In [52]:
# 종합선물세트

# 1. webdriver를 킨다.
# 2. 지역별 주유소 찾기 접속


from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get('https://www.opinet.co.kr/searRgSelect.do')

time.sleep(1)
driver.execute_script('goSubPage(0,0,99)')



# 3. sido 목록을 가져온다.

time.sleep(2)

sido = driver.find_element(By.XPATH,'//*[@id="SIDO_NM0"]')
sido_names = sido.find_elements(By.TAG_NAME,'option')

sido_list = []
for sido_name in sido_names:
    sido_list.append(sido_name.get_attribute('value'))

sido_list = sido_list[1:]


# 4. 원하는 지역으로 이동한다.
# 5. 시/군/구 목록을 가져온다.


for sido_name in sido_list:
    sido = driver.find_element(By.XPATH,'//*[@id="SIDO_NM0"]')
    sido.send_keys(sido_name)
    time.sleep(1)

    sigungu = driver.find_element(By.XPATH,'//*[@id="SIGUNGU_NM0"]')
    sigungu_names = sigungu.find_elements(By.TAG_NAME,'option')

    sigungu_list = []

    for sigungu_name in sigungu_names:
        sigungu_list.append(sigungu_name.get_attribute('value'))

    sigungu_list = sigungu_list[1:]



    # 6. 얻어온 목록으로 반복문을 수행하면서, 조회를 누르고 엑셀저장을 누른다.

    for sigungu_name in sigungu_list:
        sigungu = driver.find_element(By.XPATH,'//*[@id="SIGUNGU_NM0"]')
        time.sleep(1)
        sigungu.send_keys(sigungu_name) 
        time.sleep(1)        
        driver.find_element(By.XPATH,'//*[@id="searRgSelect"]').click()
        time.sleep(1)
        driver.find_element(By.XPATH,'//*[@id="glopopd_excel"]').click()
